In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import math

In [2]:
eye_mask_folder = "../Produced Images/Eye Masked"

In [3]:
if os.path.exists(eye_mask_folder):
    # Extract all image file names
    image_names = [
        file for file in os.listdir(eye_mask_folder)
        if os.path.isfile(os.path.join(eye_mask_folder, file))
    ]
    print("Image Names:", image_names)
else:
    print(f"The folder '{eye_mask_folder}' does not exist.")

Image Names: ['1710062483477.jpg', '1710062524242.jpg', '1710062617330.jpg', '1710063043894.jpg', '1710063172053.jpg', '1710063512010.jpg', '1710067687562.jpg', '1710067886551.jpg', '1710068440387.jpg', '1710068554866.jpg', '1710068975944.jpg', '1710069287190.jpg', '1710069549390.jpg', '1710069973175.jpg', '1710070192681.jpg', '1710070472314.jpg', '1710070755134.jpg', '1710071452062.jpg', '1710071755958.jpg', '1710072078629.jpg', '1710072405617.jpg', '1709617490220.jpg', '1709617782204.jpg', '1709618916134.jpg', '1709619661748.jpg', '1709620146694.jpg', '1709620595216.jpg', '1709621062603.jpg', '1709621461793.jpg', '1709621597176.jpg', '1709622533228.jpg', '1709622587454.jpg', '1709623125542.jpg', '1709623701628.jpg', '1709624212725.jpg', '1709624290874.jpg', '1709625264664.jpg', '1709625767525.jpg', '1709626383934.jpg', '1709626920925.jpg', '1709627337457.jpg', '1709627853782.jpg', '1709628184089.jpg', '1709628406462.jpg', '1709629128455.jpg', '1709630906630.jpg', '1709630909352.jpg',

In [14]:
type(image_names)

list

In [6]:
df1 = pd.read_excel("../Dataset Used/Blood Report_without patient.xlsx",sheet_name="left_eye")
df1.head()

,1. Take a photo of Left eye,Blood Sample Id,Unnamed: 2,Unnamed: 3,Annotated_Data,Avaiable ID,Hb Value
0,1709617490220.jpg,202403051,NaN,NaN,1709617490220.jpg,202403051.0,11.5
1,1709619661748.jpg,202403058,NaN,NaN,1709617782204.jpg,202403052.0,11.6
2,1709621461793.jpg,2024030512,NaN,NaN,1709618916134.jpg,202403056.0,11.1
3,1709624290874.jpg,2024030518,NaN,NaN,1709619661748.jpg,202403058.0,12.1
4,1709625767525.jpg,2024030522,NaN,NaN,1709620146694.jpg,202403059.0,12.2


In [7]:
filtered_df = df1[df1['Annotated_Data'].isin(image_names)]


In [8]:
filtered_df

,1. Take a photo of Left eye,Blood Sample Id,Unnamed: 2,Unnamed: 3,Annotated_Data,Avaiable ID,Hb Value
0,1709617490220.jpg,202403051,NaN,NaN,1709617490220.jpg,2.024031e+08,11.5
1,1709619661748.jpg,202403058,NaN,NaN,1709617782204.jpg,2.024031e+08,11.6
2,1709621461793.jpg,2024030512,NaN,NaN,1709618916134.jpg,2.024031e+08,11.1
3,1709624290874.jpg,2024030518,NaN,NaN,1709619661748.jpg,2.024031e+08,12.1
4,1709625767525.jpg,2024030522,NaN,NaN,1709620146694.jpg,2.024031e+08,12.2
...,...,...,...,...,...,...,...
399,1710061168367.jpg,2024031050,NaN,NaN,1710070755134.jpg,2.024031e+09,9.6
400,1710062617330.jpg,2024031053,NaN,NaN,1710071452062.jpg,2.024031e+09,11.9
401,1710063512010.jpg,2024031057,NaN,NaN,1710071755958.jpg,2.024031e+09,9.1
402,1710068440387.jpg,2024031061,NaN,NaN,1710072078629.jpg,2.024031e+09,13.9


In [28]:
filtered_columns_df = filtered_df[['Avaiable ID', 'Annotated_Data', 'Hb Value']]

In [29]:
filtered_columns_df.head()

,Avaiable ID,Annotated_Data,Hb Value
0,202403051.0,1709617490220.jpg,11.5
1,202403052.0,1709617782204.jpg,11.6
2,202403056.0,1709618916134.jpg,11.1
3,202403058.0,1709619661748.jpg,12.1
4,202403059.0,1709620146694.jpg,12.2


In [30]:
filtered_columns_df.shape

(404, 3)

In [31]:
df2 = pd.read_excel("../Dataset Used/Updated Healthworker Form Part 1_without patient identifier.xlsx") 

In [20]:
df2_filtered = df2[['Blood Sample Id', 
                            '4. Patient Sex', 
                            '2. Enter age in completed years', 
                            '8. Height (in cm)', 
                            '9. Weight (in kg)']].rename(columns={'4. Patient Sex': 'Gender', '2. Enter age in completed years': 'Age','8. Height (in cm)':'Height', '9. Weight (in kg)':'Weight'})
df2_filtered.shape

(426, 5)

In [22]:
merged_df = filtered_columns_df.merge(df2_filtered, on='Blood Sample Id', how='inner')
merged_df.shape

(416, 7)

In [23]:
filtered_columns_df.shape

(404, 3)

In [25]:
df2_filtered.shape

(426, 5)

In [26]:
print("Duplicates in filtered_columns_df:")
print(filtered_columns_df['Blood Sample Id'].duplicated().sum())

print("Duplicates in df2_filtered:")
print(df2_filtered['Blood Sample Id'].duplicated().sum())


Duplicates in filtered_columns_df:
6
Duplicates in df2_filtered:
6


In [27]:
# For filtered_columns_df
duplicates_filtered = filtered_columns_df[filtered_columns_df['Blood Sample Id'].duplicated(keep=False)]
print("Duplicate rows in filtered_columns_df:")
print(duplicates_filtered)

# For df2_filtered
duplicates_df2 = df2_filtered[df2_filtered['Blood Sample Id'].duplicated(keep=False)]
print("Duplicate rows in df2_filtered:")
print(duplicates_df2)


Duplicate rows in filtered_columns_df:
     Blood Sample Id     Annotated_Data  Hb Value
64        2024030643  1709703440550.jpg      14.8
83        2024030643  1709709970921.jpg       9.3
101       2024030681  1709715028843.jpg      11.2
122       2024030641  1709724896732.jpg       9.0
123       2024030641  1709725187979.jpg      10.5
132       2024030681  1709789179906.jpg       9.9
164       2024030750  1709796430460.jpg      12.5
183       2024030750  1709800733671.jpg      11.1
253       2024030844  1709885776309.jpg      12.4
273       2024030844  1709964363419.jpg      10.5
362       2024031055  1710053568030.jpg      12.1
381       2024031055  1710061796751.jpg      12.7
Duplicate rows in df2_filtered:
     Blood Sample Id  Gender  Age  Height  Weight
64        2024030643    Male   54   164.0    57.0
83        2024030643  Female   68   148.0    50.0
101       2024030681    Male   50   163.0    45.0
122       2024030641  Female   66   146.0    40.0
123       2024030641    Male 

In [32]:
print("Duplicates in filtered_columns_df:")
print(filtered_columns_df['Avaiable ID'].duplicated().sum())

Duplicates in filtered_columns_df:
6


In [33]:
duplicates_filtered = filtered_columns_df[filtered_columns_df['Avaiable ID'].duplicated(keep=False)]
print("Duplicate rows in filtered_columns_df:")
print(duplicates_filtered)

Duplicate rows in filtered_columns_df:
      Avaiable ID     Annotated_Data  Hb Value
90   2.024031e+09  1709712138693.jpg      14.1
92   2.024031e+09  1709712668361.jpg      10.2
93   2.024031e+09  1709712754717.jpg      14.1
96   2.024031e+09  1709713709275.jpg      10.2
128  2.024031e+09  1709726505652.jpg      13.9
129  2.024031e+09  1709726773680.jpg      13.9
176  2.024031e+09  1709799082552.jpg       8.9
180  2.024031e+09  1709799981476.jpg       8.9
246  2.024031e+09  1709883543818.jpg      10.4
247  2.024031e+09  1709884034856.jpg      10.4
380  2.024031e+09  1710061256381.jpg       8.4
382  2.024031e+09  1710061832400.jpg       8.4
